In [ ]:
"""
* this is a second version of the sandbox where i use media pipes tasks api calls instead of the older solutions version.
* It shows how to use the tasks api to read hand landmarks
*
"""

In [1]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import tensorflow as tf
from tensorflow import timestamp
from tensorflow.keras.models import load_model

In [2]:
# This is a call back function that draws the landmark results on the frame we pass it into the vision task so it knows what to do with the results
def print_result(result: vision.HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global annotated_frame
    annotated_frame = draw_landmarks_on_image(output_image.numpy_view(), result)

In [3]:
model_path = "tasks/hand_landmarker.task"
base_options = python.BaseOptions(model_asset_path=model_path) #specify where the model/task we are using for this script is and add it to the configuration
# Adding more configuration to for the task.
# here we said using the hand landmarker task, look for up to 2 hands or less, using the LIVE_STREAM mode since it is a live video feed and then print the results to the frame
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2, running_mode=vision.RunningMode.LIVE_STREAM,
                                       result_callback=print_result)
#then we create the detecor object using our configurations.
detector = vision.HandLandmarker.create_from_options(options)
annotated_frame = None # lets us detect any changes in the frame after we run detection

In [4]:
# Utility function to draw images on the screen based on detection results.
def draw_landmarks_on_image(rgb_image, detection_result):
    import numpy as np
    annotated_image = rgb_image.copy() # make a copy of the frame
    # Loop through the number of hands and create a circle on the new frame for each one at its position
    for hand_landmarks in detection_result.hand_landmarks:
        for landmark in hand_landmarks:
            x = int(landmark.x * annotated_image.shape[1])
            y = int(landmark.y * annotated_image.shape[0])
            cv2.circle(annotated_image, (x, y), 5, (0, 255, 0), -1)
    return annotated_image # new image with the landmarks drawn.

In [5]:
#Reading From a video file
cap = cv2.VideoCapture(0)

cv2.namedWindow('MediaPipe', cv2.WINDOW_NORMAL)
cv2.resizeWindow('MediaPipe', 960, 540)  # width, height

timestamp = 0
while cap.isOpened():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    try:
        #convert to RGB format
        RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #convert the image to media pipe image
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=RGB)
        # generate landmark data
        detector.detect_async(mp_image, timestamp)
        timestamp += 33

        # checks which frame to draw. if there are no landmarks detected it draws the original frame
        if annotated_frame is not None:
            cv2.imshow('MediaPipe', cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)) # here we convert it back to rbg to avoid the blue tint
            # cv2.imshow('MediaPipe', annotated_frame)
        else:
            cv2.imshow('MediaPipe', frame)
    except:
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
